### Imports

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

### Variables

In [2]:
# to store information
information = {'price': [], 'address': [], '# beds': [], '# bathrooms': [], 'sqft': [], 'description': []}


### Scraping

In [3]:
# looping through urls
for i in range(2, 21):
    # passing in url
    url = f'https://www.zillow.com/homes/for_sale/house,condo,multifamily,mobile,townhouse_type/{i}_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-124.67501706339816%2C%22east%22%3A-120.25301999308566%2C%22south%22%3A36.41890074540627%2C%22north%22%3A38.62914963315323%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A8%2C%22customRegionId%22%3A%221d2e35fe17X1-CR1va6xspgmqnpa_12pfr1%22%2C%22pagination%22%3A%7B%22currentPage%22%3A{i}%7D%7D'
    r = requests.get(url).text
    soup = bs(r, 'lxml')
    
    container_of_information = soup.find('ul', class_='photo-cards photo-cards_wow photo-cards_short')
    for card in container_of_information.find_all('li'):
        try:
            # adding description
            information['description'].append(card.find('div', class_='list-card-footer').text)

            # adding price
            information['price'].append(card.find('div', class_='list-card-price').text)

            more_details = card.find('ul', class_='list-card-details')

            # looping through list-items in ul
            for index, li in enumerate(more_details.find_all('li')):

                if index == 0: # first item is info of bed
                    information['# beds'].append(li.text)

                elif index == 1: # second item is info of bathrooms
                    information['# bathrooms'].append(li.text)

                else: # third/last info is of sqft
                    information['sqft'].append(li.text)

            # adding address
            information['address'].append(card.find('address', class_='list-card-addr').text)
        except:
            # missing values
            pass

### Viewing information

In [4]:
# checking # of samples
for element in information['sqft']:
    if 'sqft' not in element:
        information['sqft'].remove(element)
for key in information.keys():
    print(len(information[key]))

760
760
760
760
760
760


### Making DataFrame

In [5]:
df = pd.DataFrame(information)
df.head(3)

,price,address,# beds,# bathrooms,sqft,description
0,"$229,000","16 Windjammer Pl, Daly City, CA 94014",2 bds,2 ba,"1,140 sqft",Home for sale
1,"$929,000","591 Pointe Pacific APT 3, Daly City, CA 94014",3 bds,3 ba,"1,690 sqft",Condo for sale
2,"$899,000","230 N Lake Merced Hls APT 1D, San Francisco, C...",3 bds,2 ba,"1,474 sqft",Condo for sale


## Cleaning

##### price column

In [6]:
# can only replace one at a time
df['price'] = df['price'].str.replace('$', '')
df['price'] = df['price'].str.replace(',', '', 2)
df['price'] = df['price'].str.replace('Est. ', '')
df['price'] = df['price'].str.replace('+', '')
df['price'] = pd.to_numeric(df['price'])
df.head(3)

,price,address,# beds,# bathrooms,sqft,description
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2 bds,2 ba,"1,140 sqft",Home for sale
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3 bds,3 ba,"1,690 sqft",Condo for sale
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3 bds,2 ba,"1,474 sqft",Condo for sale


##### # beds column

In [7]:
df['# beds'] = df['# beds'].str.replace('bds', '').str.strip()
df['# beds'] = df['# beds'].str.replace('bd', '').str.strip()
df['# beds'] = df['# beds'].str.replace('--', '').str.strip()
df['# beds'] = df['# beds'].str.replace('Studio', '').str.strip()
df['# beds'] = df['# beds'].str.replace(',', '').str.strip()

##### bathrooms column

In [11]:
df['# bathrooms'] = df['# bathrooms'].str.replace('ba', '').str.strip()
df['# bathrooms'] = df['# bathrooms'].str.replace('--', '').str.strip()
df['# bathrooms'] = df['# bathrooms'].str.replace(',', '').str.strip()
df.head(3)

,price,address,# beds,# bathrooms,sqft,description
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2,1140,Home for sale
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3,1690,Condo for sale
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2,1474,Condo for sale


##### sqft column

In [9]:
df['sqft'] = df['sqft'].str.replace(',', '', 2)
df['sqft'] = df['sqft'].str.replace('sqft', '').str.strip()
df['sqft'] = df['sqft'].str.replace('--', '').str.strip()
df.tail(3)

,price,address,# beds,# bathrooms,sqft,description
757,799000,"1038 Pine St #B, San Francisco, CA 94109",1,1,714,Condo for sale
758,1795000,"330 8th Ave, San Francisco, CA 94118",5,2,3118,Multi-family home for sale
759,1100000,"555 4th St UNIT 429, San Francisco, CA 94107",2,2,1022,Condo for sale


##### final touch

In [12]:
# replacing all '?'s with NaN
df = df.replace('', np.NaN)

df['# beds'] = pd.to_numeric(df['# beds'])
df['# bathrooms'] = pd.to_numeric(df['# bathrooms'])
df['sqft'] = pd.to_numeric(df['sqft'])

### View cleaned data

In [13]:
df.head(25)

,price,address,# beds,# bathrooms,sqft,description
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2.0,1140.0,Home for sale
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3.0,1690.0,Condo for sale
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2.0,1474.0,Condo for sale
3,1225000,"195 Balboa Way, San Bruno, CA 94066",4.0,3.0,1780.0,House for sale
4,490681,"1875 Mission St APT 205, San Francisco, CA 94103",2.0,1.0,781.0,Condo for sale
5,3999888,"123 Woodward Ave, Sausalito, CA 94965",5.0,6.0,5242.0,House for sale
6,267548,"1695 18th St SUITE 409, San Francisco, CA 94107",NaN,1.0,636.0,Condo for sale
7,675000,"33 W Harbor Dr, Sausalito, CA 94965",1.0,1.0,829.0,Condo for sale
8,679000,"1264A Golden Gate Ave, San Francisco, CA 94115",3.0,1.0,874.0,Condo for sale
9,988000,"411 Talbot Ave, Pacifica, CA 94044",4.0,2.0,1750.0,House for sale


## Questions

In [14]:
df.head()

,price,address,# beds,# bathrooms,sqft,description
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2.0,1140.0,Home for sale
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3.0,1690.0,Condo for sale
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2.0,1474.0,Condo for sale
3,1225000,"195 Balboa Way, San Bruno, CA 94066",4.0,3.0,1780.0,House for sale
4,490681,"1875 Mission St APT 205, San Francisco, CA 94103",2.0,1.0,781.0,Condo for sale


### Q1: What are the types of houses and how many are there of each?

In [15]:
df['description'].value_counts()

Condo for sale                266
House for sale                242
Contingent                    113
Multi-family home for sale     30
For sale by owner               9
Townhouse for sale              8
Home for sale                   8
New construction                3
Auction                         1
Name: description, dtype: int64

#### A1: check above

### Q2: Find the average price for each different # of beds

In [16]:
df.groupby('# beds').mean()['price']

# beds
1.0     7.015773e+05
2.0     1.103997e+06
3.0     1.767774e+06
4.0     3.442963e+06
5.0     4.292303e+06
6.0     9.934111e+06
7.0     6.272750e+06
8.0     1.472167e+07
9.0     7.797500e+06
11.0    2.690000e+06
Name: price, dtype: float64

#### A2: check above

### Q3: Find the average price of house for each thousand sqft

In [17]:
df['Thousands'] = df['sqft'] // 1000
df.head()

,price,address,# beds,# bathrooms,sqft,description,Thousands
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2.0,1140.0,Home for sale,1.0
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3.0,1690.0,Condo for sale,1.0
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2.0,1474.0,Condo for sale,1.0
3,1225000,"195 Balboa Way, San Bruno, CA 94066",4.0,3.0,1780.0,House for sale,1.0
4,490681,"1875 Mission St APT 205, San Francisco, CA 94103",2.0,1.0,781.0,Condo for sale,0.0


In [18]:
print(df.groupby('Thousands').mean()['price'])
df.drop('Thousands', axis=1, inplace=True)

Thousands
0.0     7.053811e+05
1.0     1.253184e+06
2.0     2.068729e+06
3.0     3.426856e+06
4.0     6.145696e+06
5.0     9.302052e+06
6.0     1.144143e+07
7.0     2.299000e+07
9.0     8.990000e+06
10.0    2.290000e+07
11.0    2.166500e+07
13.0    1.780000e+07
15.0    2.288000e+07
Name: price, dtype: float64


#### A3: above

### Q4: Find the average number of total rooms based off sqft

In [19]:
df['total rooms'] = df['# beds'] + df['# bathrooms']
df['sqft in thousands'] = df['sqft'] // 1000
df.head()

,price,address,# beds,# bathrooms,sqft,description,total rooms,sqft in thousands
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2.0,1140.0,Home for sale,4.0,1.0
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3.0,1690.0,Condo for sale,6.0,1.0
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2.0,1474.0,Condo for sale,5.0,1.0
3,1225000,"195 Balboa Way, San Bruno, CA 94066",4.0,3.0,1780.0,House for sale,7.0,1.0
4,490681,"1875 Mission St APT 205, San Francisco, CA 94103",2.0,1.0,781.0,Condo for sale,3.0,0.0


In [20]:
df.groupby('sqft in thousands').mean()['total rooms']

sqft in thousands
0.0      2.670000
1.0      4.593553
2.0      6.766055
3.0      8.220930
4.0      8.913043
5.0     10.500000
6.0     10.142857
7.0     12.900000
9.0     16.000000
10.0    13.750000
11.0    16.666667
13.0    18.000000
15.0    16.000000
Name: total rooms, dtype: float64

#### A4: check above

### Q5a: Which cities have the most houses and the least? 
### Q5b: And how many different cities are there?

In [61]:
df['city'] = df['address'].str.split(",").str[1]
df.head()

,price,address,# beds,# bathrooms,sqft,description,total rooms,sqft in thousands,city
0,229000,"16 Windjammer Pl, Daly City, CA 94014",2.0,2.0,1140.0,Home for sale,4.0,1.0,Daly City
1,929000,"591 Pointe Pacific APT 3, Daly City, CA 94014",3.0,3.0,1690.0,Condo for sale,6.0,1.0,Daly City
2,899000,"230 N Lake Merced Hls APT 1D, San Francisco, C...",3.0,2.0,1474.0,Condo for sale,5.0,1.0,San Francisco
3,1225000,"195 Balboa Way, San Bruno, CA 94066",4.0,3.0,1780.0,House for sale,7.0,1.0,San Bruno
4,490681,"1875 Mission St APT 205, San Francisco, CA 94103",2.0,1.0,781.0,Condo for sale,3.0,0.0,San Francisco


In [63]:
df['city'].value_counts().head(2)

 San Francisco    505
 Daly City         33
Name: city, dtype: int64

In [64]:
df['city'].value_counts().tail(2)

 Belvedere    2
 305          1
Name: city, dtype: int64

#### A5a: San Francisco has the most homes. Belvedere has least # of homes.

In [66]:
df['city'].nunique()

28

#### A5b: There are 28 cities in this dataset

### Q6a: Average cost per city
### Q6b: Sum of all houses in each city
### Q6c: What type of home is typically most expensive

In [70]:
df.groupby('city').mean()['price'].head(3)

city
 305                  5.990000e+05
 Belvedere            7.195000e+06
 Belvedere Tiburon    8.356222e+06
Name: price, dtype: float64

#### Q6a: check above

In [71]:
df.groupby('city').sum()['price']

city
 305                        599000
 Belvedere                14390000
 Belvedere Tiburon        75206000
 Brisbane                  4400000
 Burlingame               32285000
 Corte Madera              6853000
 Daly City                31701098
 El Granada                5444000
 Fairfax                  11162000
 Greenbrae                 1171000
 Half Moon Bay            12437000
 Hillsborough             62061998
 Kentfield                 6473000
 Larkspur                 10079900
 Mill Valley              16482999
 Millbrae                  9898841
 Montara                   2499000
 Moss Beach               15689000
 Pacifica                 23750000
 Ross                     57300000
 San Anselmo              14449800
 San Bruno                18250776
 San Francisco          1043204064
 San Mateo                21221464
 San Rafael               13874000
 Sausalito                42585888
 South San Francisco       7296039
 Tiburon                  54402995
Name: price, dt

#### Q6b: check above

In [74]:
df.groupby('description').mean()['price']

description
Auction                       1.109372e+06
Condo for sale                1.310877e+06
Contingent                    1.404996e+06
For sale by owner             1.376741e+06
Home for sale                 6.528750e+05
House for sale                3.392131e+06
Multi-family home for sale    1.964632e+06
New construction              1.198667e+06
Townhouse for sale            1.028711e+06
Name: price, dtype: float64

#### Q6c: 'House for sale' cost more on average compared to other houses with different descriptions